Auteur: Michaël Leroy


Modification du traitement pour la création des edges graph communes, essai sur geodataframe pour profiter de la vectorisation:
        testé jusqu'a la recherche de paths... qui n'est pas vect et du coup hyper long (j'ai killé au bout de 3h)

    Création geodataframe Edges:

        calcul des edges attributes:
            - shortest path depuis le graph routier => distance, durée
            - geométrie: somme des LineString
            - length : somme des longeurs de chaque tronçons en mètres

[+] A faire:
        
        - prolonger les geometries des linestrings routes jusqu'au points centraux communes si on veut se servir des tracés sur une carte, sinon pas grave.

[-] Cons:
        
        40 min pour 250 nodes et 1219 edges

        -> temps estimé pour 170188 edges ~76h


In [13]:
class CFG:
    area = [
        'Auvergne-Rhône-Alpes',
    'Bourgogne-Franche-Comté',
    'Brittany',
    'Centre-Val de Loire',
    'Grand Est',
    'Hauts-de-France',
    'Île-de-France',
    'Normandie',
    'Nouvelle-Aquitaine',
    'Occitanie',
    'Pays de la Loire',
    'Provence-Alpes-Côte d\'Azur'
]
    area_name = 'france'      #'_'.join(area)

    

    bornes = False

    viz = True

    roads = (
        '|motorway'
        '|motorway_link'
        '|trunk'
        '|trunk_link'
        '|primary'
        '|primary_link'
        '|secondary'
        '|secondary_link'
        '|tertiary'
         '|tertiary_link'
    )

        # '|primary'
        # '|motorway_link'
        
        # '|primary_link'

        # '|secondary'
        # '|secondary_link'
        
        
        #  '|tertiary'
        #  '|tertiary_link'

    data_path = './osm_datas/'

import joblib
joblib.dump(CFG.area, 'CFG.pkl')    

import os
os.environ['USE_PYGEOS'] = '0'

%load_ext autoreload
%autoreload


%aimport OSM_helper_functions
%aimport OCM_helper_functions

import os
os.environ['USE_PYGEOS'] = '0'
import geopandas as gpd
import osmnx as ox
import networkx as nx
from tqdm.notebook import tqdm
from OSM_helper_functions import *
from OCM_helper_functions import *


ox.settings.log_console=True
ox.settings.use_cache=True
''' To retrieve http data from OSMnx and cache them locally
    before building graph. Raise and exception if requests
    are all done. intercept then set to False and retry.'''
# ox.settings.cache_only_mode=True 

Administratives = {
    'Auvergne-Rhône-Alpes': ['Ain', 'Allier', 'Ardèche', 'Cantal', 'Drôme', 'Isère', 'Loire', 'Haute-Loire', 'Puy-de-Dôme', 'Rhône', 'Savoie', 'Haute-Savoie'],
    'Bourgogne-Franche-Comté': ['Côte-d\'Or', 'Doubs', 'Jura', 'Nièvre', 'Haute-Saône', 'Saône-et-Loire', 'Yonne', 'Territoire de Belfort'],
    'Brittany': ['Côtes-d\'Armor', 'Finistère', 'Ille-et-Vilaine', 'Morbihan'],
    'Centre-Val de Loire': ['Cher', 'Eure-et-Loir', 'Indre', 'Indre-et-Loire', 'Loir-et-Cher', 'Loiret'],
    # 'Corse': ['Corse-du-Sud', 'Haute-Corse'],
    'Grand Est': ['Ardennes', 'Aube', 'Marne', 'Haute-Marne', 'Meurthe-et-Moselle', 'Meuse', 'Moselle', 'Bas-Rhin', 'Haut-Rhin', 'Vosges'],
    'Hauts-de-France': ['Aisne', 'Nord', 'Oise', 'Pas-de-Calais', 'Somme'],
    'Île-de-France': ['Paris', 'Seine-et-Marne', 'Yvelines', 'Essonne', 'Hauts-de-Seine', 'Seine-Saint-Denis', 'Val-de-Marne', 'Val-d\'Oise'],
    'Normandie': ['Calvados', 'Eure', 'Manche', 'Orne', 'Seine-Maritime'],
    'Nouvelle-Aquitaine': ['Charente', 'Charente-Maritime', 'Corrèze', 'Creuse', 'Dordogne', 'Gironde', 'Landes', 'Lot-et-Garonne', 'Pyrénées-Atlantiques', 'Deux-Sèvres', 'Vienne', 'Haute-Vienne'],
    'Occitanie': ['Ariège', 'Aude', 'Aveyron', 'Gard', 'Haute-Garonne', 'Gers', 'Hérault', 'Lot', 'Lozère', 'Hautes-Pyrénées', 'Pyrénées-Orientales', 'Tarn', 'Tarn-et-Garonne'],
    'Pays de la Loire': ['Loire-Atlantique', 'Maine-et-Loire', 'Mayenne', 'Sarthe', 'Vendée'],
    'Provence-Alpes-Côte d\'Azur': ['Alpes-de-Haute-Provence', 'Hautes-Alpes', 'Alpes-Maritimes', 'Bouches-du-Rhône', 'Var', 'Vaucluse']
}

Regions = [
    'Auvergne-Rhône-Alpes',
    'Bourgogne-Franche-Comté',
    'Brittany',
    'Centre-Val de Loire',
    # 'Corse',
    'Grand Est',
    'Hauts-de-France',
    'Île-de-France',
    'Normandie',
    'Nouvelle-Aquitaine',
    'Occitanie',
    'Pays de la Loire',
    'Provence-Alpes-Côte d\'Azur'
]

Name2num = {
    'Ain': '01',
    'Aisne': '02',
    'Allier': '03',
    'Alpes-de-Haute-Provence': '04',
    'Hautes-Alpes': '05',
    'Alpes-Maritimes': '06',
    'Ardèche': '07',
    'Ardennes': '08',
    'Ariège': '09',
    'Aube': '10',
    'Aude': '11',
    'Aveyron': '12',
    'Bouches-du-Rhône': '13',
    'Calvados': '14',
    'Cantal': '15',
    'Charente': '16',
    'Charente-Maritime': '17',
    'Cher': '18',
    'Corrèze': '19',
    # 'Corse-du-Sud': '2A',
    # 'Haute-Corse': '2B',
    'Côte-d\'Or': '21',
    'Côtes-d\'Armor': '22',
    'Creuse': '23',
    'Dordogne': '24',
    'Doubs': '25',
    'Drôme': '26',
    'Eure': '27',
    'Eure-et-Loir': '28',
    'Finistère': '29',
    'Gard': '30',
    'Haute-Garonne': '31',
    'Gers': '32',
    'Gironde': '33',
    'Hérault': '34',
    'Ille-et-Vilaine': '35',
    'Indre': '36',
    'Indre-et-Loire': '37',
    'Isère': '38',
    'Jura': '39',
    'Landes': '40',
    'Loir-et-Cher': '41',
    'Loire': '42',
    'Haute-Loire': '43',
    'Loire-Atlantique': '44',
    'Loiret': '45',
    'Lot': '46',
    'Lot-et-Garonne': '47',
    'Lozère': '48',
    'Maine-et-Loire': '49',
    'Manche': '50',
    'Marne': '51',
    'Haute-Marne': '52',
    'Mayenne': '53',
    'Meurthe-et-Moselle': '54',
    'Meuse': '55',
    'Morbihan': '56',
    'Moselle': '57',
    'Nièvre': '58',
    'Nord': '59',
    'Oise': '60',
    'Orne': '61',
    'Pas-de-Calais': '62',
    'Puy-de-Dôme': '63',
    'Pyrénées-Atlantiques': '64',
    'Hautes-Pyrénées': '65',
    'Pyrénées-Orientales': '66',
    'Bas-Rhin': '67',
    'Haut-Rhin': '68',
    'Rhône': '69',
    'Haute-Saône': '70',
    'Saône-et-Loire': '71',
    'Sarthe': '72',
    'Savoie': '73',
    'Haute-Savoie': '74',
    'Paris': '75',
    'Seine-Maritime': '76',
    'Seine-et-Marne': '77',
    'Yvelines': '78',
    'Deux-Sèvres': '79',
    'Somme': '80',
    'Tarn': '81',
    'Tarn-et-Garonne': '82',
    'Var': '83',
    'Vaucluse': '84',
    'Vendée': '85',
    'Vienne': '86',
    'Haute-Vienne': '87',
    'Vosges': '88',
    'Yonne': '89',
    'Territoire de Belfort': '90',
    'Essonne': '91',
    'Hauts-de-Seine': '92',
    'Seine-Saint-Denis': '93',
    'Val-de-Marne': '94',
    'Val-d\'Oise': '95'
}


AREA = []
for area in CFG.area:
    AREA.extend(Administratives[area] )
# AREA

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [14]:
# data_path = './datas'
# file_name = 'decoupage_administratif'


# communes = gpd.read_feather(os.path.join(data_path,f'[{CFG.area_name}]{file_name}_osmid_centre.feather'))

# communes.crs

In [15]:
# metro = Name2num.values()
# communes = communes.query("dep in @metro")


# # MANUAL FIX FOR MISSING OSMID ON Nominatim resquests failures
# communes.loc[5228 , 'osmid'] = 555489493
# communes.loc[5229 , 'osmid'] = 125363211
# communes.loc[10916, 'osmid'] = 9096959355
# communes.loc[34954, 'osmid'] = 41978755

# # Test on sample
# # deps = ['56']  #+['51','52','08','10']
# # communes = communes.query("dep in @deps")

# communes['osmid'] = communes['osmid'].astype(int)

# display(communes.head(1))
# display(communes.info())
# communes.shape

In [16]:

Nodes = joblib.load(os.path.join(CFG.data_path, f'[{CFG.area_name}]Gnodes_communes.pkl'))
Edges = joblib.load(os.path.join(CFG.data_path, f'[{CFG.area_name}]Gedges_communes.pkl'))
# G_communes = ox.graph_from_gdfs(Nodes, Edges)


In [17]:
Nodes.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
Index: 34466 entries, 2531584 to 2741108
Data columns (total 10 columns):
 #   Column    Non-Null Count  Dtype   
---  ------    --------------  -----   
 0   y         34466 non-null  float64 
 1   x         34466 non-null  float64 
 2   codgeo    34466 non-null  object  
 3   area      34466 non-null  object  
 4   sleeping  15412 non-null  float64 
 5   visit     17923 non-null  float64 
 6   fooding   6616 non-null   float64 
 7   working   6950 non-null   float64 
 8   vehicule  4166 non-null   float64 
 9   geometry  34466 non-null  geometry
dtypes: float64(7), geometry(1), object(2)
memory usage: 3.9+ MB


In [18]:
Edges.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
MultiIndex: 205450 entries, (2531584, 2176035, 0) to (2741108, 2740963, 0)
Data columns (total 1 columns):
 #   Column    Non-Null Count   Dtype   
---  ------    --------------   -----   
 0   geometry  205450 non-null  geometry
dtypes: geometry(1)
memory usage: 3.9 MB


In [19]:
Nodes_routier = joblib.load( os.path.join(CFG.data_path, f'[{CFG.area_name}]Gnodes.pkl'))
Graph_routier = ox.graph_from_gdfs(
    Nodes_routier, 
    joblib.load( os.path.join(CFG.data_path, f'[{CFG.area_name}]Gedges.pkl'))
    )

In [20]:
display(Nodes_routier.info(),Nodes_routier.head(1))

<class 'geopandas.geodataframe.GeoDataFrame'>
Int64Index: 522987 entries, 27174280 to 10907606787
Data columns (total 6 columns):
 #   Column        Non-Null Count   Dtype   
---  ------        --------------   -----   
 0   y             522987 non-null  float64 
 1   x             522987 non-null  float64 
 2   street_count  522987 non-null  int64   
 3   highway       21236 non-null   object  
 4   ref           4152 non-null    object  
 5   geometry      522987 non-null  geometry
dtypes: float64(2), geometry(1), int64(1), object(2)
memory usage: 44.1+ MB


None

,y,x,street_count,highway,ref,geometry
osmid,,,,,,
27174280,45.612495,5.609986,2,NaN,NaN,POINT (5.60999 45.61249)


In [21]:
for u, v,  d in Graph_routier.edges(data=True):
    print(u, v, d)

    break

27174280 27174283 {'osmid': 8410390, 'lanes': '2', 'ref': 'D 592', 'name': 'Route de Belley', 'highway': 'secondary', 'oneway': False, 'reversed': False, 'length': 634.682, 'speed_kph': 62.2, 'travel_time': 36.7, 'geometry': <LINESTRING (5.61 45.612, 5.61 45.613, 5.61 45.615, 5.611 45.618, 5.611 45.618)>}


In [22]:
Nodes.index = Nodes.index.astype(int)

display(Nodes.info(),Nodes.head(5))

<class 'geopandas.geodataframe.GeoDataFrame'>
Int64Index: 34466 entries, 2531584 to 2741108
Data columns (total 10 columns):
 #   Column    Non-Null Count  Dtype   
---  ------    --------------  -----   
 0   y         34466 non-null  float64 
 1   x         34466 non-null  float64 
 2   codgeo    34466 non-null  object  
 3   area      34466 non-null  object  
 4   sleeping  15412 non-null  float64 
 5   visit     17923 non-null  float64 
 6   fooding   6616 non-null   float64 
 7   working   6950 non-null   float64 
 8   vehicule  4166 non-null   float64 
 9   geometry  34466 non-null  geometry
dtypes: float64(7), geometry(1), object(2)
memory usage: 2.9+ MB


None

,y,x,codgeo,area,sleeping,visit,fooding,working,vehicule,geometry
osmid,,,,,,,,,,
2531584,43.461438,-0.819766,64312,POLYGON ((-0.832953105776137 43.44755970632465...,2.0,NaN,NaN,NaN,NaN,POINT (-0.81977 43.46144)
279376,47.730421,0.777490,41255,POLYGON ((0.7426134063245691 47.72901158483156...,NaN,1.0,NaN,NaN,NaN,POINT (0.77749 47.73042)
1022241,47.129554,6.693006,25421,POLYGON ((6.710196593401414 47.130558839705444...,1.0,NaN,1.0,NaN,NaN,POINT (6.69301 47.12955)
153920,49.703376,4.943341,08409,"POLYGON ((4.968185038340734 49.70249532252551,...",20.0,20.0,19.0,3.0,NaN,POINT (4.94334 49.70338)
111919,44.922000,3.257620,48046,POLYGON ((3.2485359975552894 44.93934468866044...,NaN,2.0,NaN,NaN,NaN,POINT (3.25762 44.92200)


In [23]:
# ox.projection.project_gdf(_edges, to_crs=2154)
# tmja_84 = ox.projection.project_gdf(tmja, to_crs='epsg:4326', to_latlong=False)
# ox.projection.project_graph(G, to_crs=None)

In [24]:
temp = Nodes[['x','y','geometry']].copy()
temp['geometry'] = [ Point(x,y) for x,y in zip(temp.x,temp.y)]
temp = ox.projection.project_gdf(temp, to_crs=2154)

temp

,x,y,geometry
osmid,,,
2531584,-0.819766,43.461438,POINT (390856.453 6269961.045)
279376,0.777490,47.730421,POINT (533425.047 6739014.384)
1022241,6.693006,47.129554,POINT (979843.569 6676465.908)
153920,4.943341,49.703376,POINT (840257.764 6957763.074)
111919,3.257620,44.922000,POINT (720328.523 6424788.989)
...,...,...,...
71694,4.906740,45.213600,POINT (849662.530 6458948.860)
5827802,7.610694,48.722684,POINT (1038980.609 6856849.884)
2573037,4.566575,44.301186,POINT (824976.968 6357035.589)


In [25]:
temp['x_proj'] = temp['geometry'].apply(lambda geom: geom.coords[0][0])
temp['y_proj'] = temp['geometry'].apply(lambda geom: geom.coords[0][1])

temp

,x,y,geometry,x_proj,y_proj
osmid,,,,,
2531584,-0.819766,43.461438,POINT (390856.453 6269961.045),3.908565e+05,6.269961e+06
279376,0.777490,47.730421,POINT (533425.047 6739014.384),5.334250e+05,6.739014e+06
1022241,6.693006,47.129554,POINT (979843.569 6676465.908),9.798436e+05,6.676466e+06
153920,4.943341,49.703376,POINT (840257.764 6957763.074),8.402578e+05,6.957763e+06
111919,3.257620,44.922000,POINT (720328.523 6424788.989),7.203285e+05,6.424789e+06
...,...,...,...,...,...
71694,4.906740,45.213600,POINT (849662.530 6458948.860),8.496625e+05,6.458949e+06
5827802,7.610694,48.722684,POINT (1038980.609 6856849.884),1.038981e+06,6.856850e+06
2573037,4.566575,44.301186,POINT (824976.968 6357035.589),8.249770e+05,6.357036e+06


In [26]:

Nodes['closest_routier'] = ox.distance.nearest_nodes(
    ox.projection.project_graph(Graph_routier, to_crs=2154),
    temp.x_proj,
    temp.y_proj
    )

In [27]:
from shapely import MultiLineString, ops


In [28]:
Edges.reset_index(inplace=True)
Edges.sample(2)

,u,v,key,geometry
16162,122588,123403,0,"LINESTRING (1.63369 44.36667, 1.64167 44.33732)"
142389,1342815,1252279,0,"LINESTRING (6.62786 48.75161, 6.64832 48.76236)"


In [29]:
mapper = dict(zip(Nodes.index.astype(int),Nodes.closest_routier))
Edges['ox_src'] = Edges['u'].astype(int).map(mapper)
Edges['ox_trgt'] = Edges['v'].astype(int).map(mapper)
Edges['src_trgt'] = tuple(zip(Edges.ox_src, Edges.ox_trgt))

In [30]:
Edges.sample(1)

,u,v,key,geometry,ox_src,ox_trgt,src_trgt
191078,124345,9135933,0,"LINESTRING (1.60963 44.93733, 1.59646 44.98802)",543861775,25634025,"(543861775, 25634025)"


In [31]:
Edges = Edges.drop(['ox_src','ox_trgt'], axis=1)

del Nodes_routier

In [32]:
import numpy as np
Edges['path'] = np.nan

In [33]:
Edges.set_index(['u','v','key'])

geometry  \
u       v       key                                                      
2531584 2176035 0    LINESTRING (-0.81977 43.46144, -0.84879 43.43624)   
        167704  0    LINESTRING (-0.81977 43.46144, -0.77384 43.48750)   
        1380951 0    LINESTRING (-0.81977 43.46144, -0.79568 43.44181)   
        2531635 0    LINESTRING (-0.81977 43.46144, -0.83523 43.49858)   
279376  279375  0      LINESTRING (0.77749 47.73042, 0.73910 47.74537)   
...                                                                ...   
2741108 5811913 0      LINESTRING (3.50709 48.86596, 3.48873 48.89115)   
        2742275 0      LINESTRING (3.50709 48.86596, 3.60070 48.80919)   
        1769723 0      LINESTRING (3.50709 48.86596, 3.53820 48.87088)   
        2742573 0      LINESTRING (3.50709 48.86596, 3.53126 48.79645)   
        2740963 0      LINESTRING (3.50709 48.86596, 3.49967 48.85079)   

                                     src_trgt  path  
u       v       key                                  
2531584 2176035 0      (859309184, 504799281)   NaN  
        167704  0     (859309184, 3957848575)   NaN  
        1380951 0     (859309184, 1976671808)   NaN  
        2531635 0     (859309184, 1975118728)   NaN  
279376  279375  0     (281892451, 1372889494)   NaN  
...                                       ...   ...  
2741108 5811913 0    (3427753813, 3737538194)   NaN  
        2742275 0    (3427753813, 1296924098)   NaN  
        1769723 0    (3427753813, 5928930665)   NaN  
        2742573 0    (3427753813, 1092502737)   NaN  
        2740963 0    (3427753813, 8335004268)   NaN  

[205450 rows x 3 columns]

In [34]:
(
Edges
 .set_index(['u','v'])
 .to_feather(os.path.join(CFG.data_path, f'[{CFG.area_name}]Gedges_communes_0_v(ox).feather'))
)

-------------------- 